In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import statsmodels.api as sm

In [4]:
# Leitura do data set
df = pd.read_csv('water_consumption - water_consumption.csv')
df.head()

,initial,final,price,month,year,block,apartment
0,3535.0,3565.0,464.1,7.0,2023,A,11
1,3375.0,3402.0,380.1,7.0,2023,A,12
2,3620.0,3651.0,492.1,7.0,2023,A,21
3,4681.0,4707.0,352.1,NaN,2023,A,22
4,2400.0,2425.0,324.1,NaN,2023,A,31


In [5]:
print("Estatísticas descritivas iniciais:")
print("=================================")
df.describe() # O uso do describe faz com que cada coluna numérica tenha alguns dados estatísticos

Estatísticas descritivas iniciais:


,initial,final,price,month,year,apartment
count,387.000000,387.000000,390.000000,387.0,390.0,390.000000
mean,2692.824289,2713.002584,271.202615,7.0,2023.0,116.269231
std,1043.806818,1054.557620,224.042885,0.0,0.0,64.432399
min,0.000000,0.000000,71.700000,7.0,2023.0,11.000000
25%,2024.500000,2046.500000,127.900000,7.0,2023.0,62.000000
50%,2637.000000,2650.000000,184.100000,7.0,2023.0,113.000000
75%,3372.500000,3390.000000,352.100000,7.0,2023.0,171.000000
max,7211.000000,7240.000000,2042.480000,7.0,2023.0,272.000000


In [6]:
# Ao utilizarmos o value_counts() um dicionário é gerado
df["block"].value_counts()

block
D    80
E    80
A    54
C    44
B    44
F    44
G    44
Name: count, dtype: int64

In [7]:
# One-Hot Encoding da Variável 'block'
df_copy = df.copy()
block_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

for block in block_list:
    df_copy[block] = df_copy['block'].apply( lambda x: 1 if x == block else 0 )

df_copy = df_copy.drop('block', axis=1)
df_copy

,initial,final,price,month,year,apartment,A,B,C,D,E,F,G
0,3535.0,3565.0,464.1,7.0,2023,11,1,0,0,0,0,0,0
1,3375.0,3402.0,380.1,7.0,2023,12,1,0,0,0,0,0,0
2,3620.0,3651.0,492.1,7.0,2023,21,1,0,0,0,0,0,0
3,4681.0,4707.0,352.1,NaN,2023,22,1,0,0,0,0,0,0
4,2400.0,2425.0,324.1,NaN,2023,31,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,2576.0,2586.0,71.7,7.0,2023,202,0,0,0,0,0,0,1
386,2142.0,2163.0,212.1,7.0,2023,211,0,0,0,0,0,0,1
387,2682.0,2703.0,212.1,7.0,2023,212,0,0,0,0,0,0,1
388,1992.0,2028.0,632.1,7.0,2023,221,0,0,0,0,0,0,1


In [8]:
# Remoção de valores nulos
df_copy.dropna(inplace=True)
df_copy.reset_index(drop=True, inplace=True)
df_copy

,initial,final,price,month,year,apartment,A,B,C,D,E,F,G
0,3535.0,3565.0,464.10,7.0,2023,11,1,0,0,0,0,0,0
1,3375.0,3402.0,380.10,7.0,2023,12,1,0,0,0,0,0,0
2,3620.0,3651.0,492.10,7.0,2023,21,1,0,0,0,0,0,0
3,4830.0,4862.0,520.10,7.0,2023,41,1,0,0,0,0,0,0
4,3525.0,3543.0,161.62,7.0,2023,42,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2576.0,2586.0,71.70,7.0,2023,202,0,0,0,0,0,0,1
377,2142.0,2163.0,212.10,7.0,2023,211,0,0,0,0,0,0,1
378,2682.0,2703.0,212.10,7.0,2023,212,0,0,0,0,0,0,1
379,1992.0,2028.0,632.10,7.0,2023,221,0,0,0,0,0,0,1


In [9]:
# Remoção de valores initial ou final nulos
df_copy = df_copy[(df_copy["initial"] != 0) & (df_copy["final"] != 0)]
df_copy

,initial,final,price,month,year,apartment,A,B,C,D,E,F,G
0,3535.0,3565.0,464.10,7.0,2023,11,1,0,0,0,0,0,0
1,3375.0,3402.0,380.10,7.0,2023,12,1,0,0,0,0,0,0
2,3620.0,3651.0,492.10,7.0,2023,21,1,0,0,0,0,0,0
3,4830.0,4862.0,520.10,7.0,2023,41,1,0,0,0,0,0,0
4,3525.0,3543.0,161.62,7.0,2023,42,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2576.0,2586.0,71.70,7.0,2023,202,0,0,0,0,0,0,1
377,2142.0,2163.0,212.10,7.0,2023,211,0,0,0,0,0,0,1
378,2682.0,2703.0,212.10,7.0,2023,212,0,0,0,0,0,0,1
379,1992.0,2028.0,632.10,7.0,2023,221,0,0,0,0,0,0,1


In [10]:
# Cálculo da Variável Alvo 'consume'
df_copy['consume'] = df_copy['final'] - df_copy['initial']
df_copy.describe()

/tmp/ipykernel_2377/2693562383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy['consume'] = df_copy['final'] - df_copy['initial']


,initial,final,price,month,year,apartment,A,B,C,D,E,F,G,consume
count,380.000000,380.000000,380.000000,380.0,380.0,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000,380.000000
mean,2686.063158,2706.592105,268.809632,7.0,2023.0,117.421053,0.118421,0.113158,0.115789,0.210526,0.210526,0.115789,0.115789,20.528947
std,1033.240909,1038.171413,223.117759,0.0,0.0,64.357647,0.323532,0.317203,0.320394,0.408220,0.408220,0.320394,0.320394,10.488929
min,9.000000,26.000000,71.700000,7.0,2023.0,11.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-18.000000
25%,2009.000000,2035.500000,127.900000,7.0,2023.0,62.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000
50%,2633.000000,2649.500000,184.100000,7.0,2023.0,121.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
75%,3364.250000,3385.500000,352.100000,7.0,2023.0,171.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000
max,7211.000000,7240.000000,2042.480000,7.0,2023.0,272.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,83.000000


In [11]:
# Consumo negativo IMPOSSÍVEL
df_copy = df_copy[(df_copy["consume"] > 0)]
df_copy

,initial,final,price,month,year,apartment,A,B,C,D,E,F,G,consume
0,3535.0,3565.0,464.10,7.0,2023,11,1,0,0,0,0,0,0,30.0
1,3375.0,3402.0,380.10,7.0,2023,12,1,0,0,0,0,0,0,27.0
2,3620.0,3651.0,492.10,7.0,2023,21,1,0,0,0,0,0,0,31.0
3,4830.0,4862.0,520.10,7.0,2023,41,1,0,0,0,0,0,0,32.0
4,3525.0,3543.0,161.62,7.0,2023,42,1,0,0,0,0,0,0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,2576.0,2586.0,71.70,7.0,2023,202,0,0,0,0,0,0,1,10.0
377,2142.0,2163.0,212.10,7.0,2023,211,0,0,0,0,0,0,1,21.0
378,2682.0,2703.0,212.10,7.0,2023,212,0,0,0,0,0,0,1,21.0
379,1992.0,2028.0,632.10,7.0,2023,221,0,0,0,0,0,0,1,36.0


In [12]:
# 10. Modelo Inicial de Regressão Linear
model = LinearRegression()
x = df_copy[['price','month','year','apartment','A','B','C','D','E','F','G']]
y = df_copy['consume']

model.fit(x, y)
print(model.score(x, y))

x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

0.9255250907188203
                            OLS Regression Results                            
Dep. Variable:                consume   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     551.5
Date:                Wed, 27 Aug 2025   Prob (F-statistic):          4.63e-195
Time:                        06:56:39   Log-Likelihood:                -867.15
No. Observations:                 364   AIC:                             1752.
Df Residuals:                     355   BIC:                             1787.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
price          0.0417      0.001 

In [13]:
# 11. Seleção de Variáveis Significativas
model = LinearRegression()
x = df_copy[['price','month','year']]
y = df_copy['consume']

model.fit(x, y)
print(model.score(x, y))

x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

0.9248804266733907
                            OLS Regression Results                            
Dep. Variable:                consume   R-squared:                       0.925
Model:                            OLS   Adj. R-squared:                  0.925
Method:                 Least Squares   F-statistic:                     4457.
Date:                Wed, 27 Aug 2025   Prob (F-statistic):          1.41e-205
Time:                        06:57:09   Log-Likelihood:                -868.72
No. Observations:                 364   AIC:                             1741.
Df Residuals:                     362   BIC:                             1749.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
price          0.0415      0.001 